In [1]:
import yfinance as yf
import pandas as pd
import time

### Define Parameters

In [2]:
# Project time frame
start_date = "2020-01-01"
end_date = "2025-05-01"

# Benchmark index
benchmark_symbol = "^GSPC"

# Top 10 symbols from each sector (replace with your actual tickers if needed)
sectors = {
    "Technology": ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'META', 'TSLA', 'NFLX', 'NVDA', 'INTC', 'AMD'],
    # "Communication": ["GOOGL", "META", "NFLX", "TMUS", "DIS", "VZ", "T", "CHTR", "CMCSA", "WBD"],
    # "Health": ["JNJ", "PFE", "ABBV", "LLY", "MRK", "TMO", "BMY", "UNH", "ABT", "CVS"],
    # "Financials": ["JPM", "BAC", "WFC", "C", "GS", "MS", "AXP", "USB", "BK", "SCHW"],
    # "Defensive": ["PG", "KO", "PEP", "WMT", "COST", "CL", "MO", "KMB", "MDLZ", "KR"],
    # "Cyclical": ["AMZN", "HD", "LOW", "MCD", "NKE", "SBUX", "BKNG", "TGT", "EBAY", "GM"],
    # "Property": ["PLD", "AMT", "CCI", "EQIX", "DLR", "SPG", "O", "PSA", "VTR", "EXR"],
    "Benchmark": [benchmark_symbol]
}

# Flatten list of all tickers
all_symbols = [symbol for group in sectors.values() for symbol in group]

### Download Historical Stock Data with Sector Labels

In [3]:
def download_by_sector_batch(sectors_dict, start, end, interval="1wk"):
    all_data = []

    for sector, symbols in sectors_dict.items():
        print(f"Downloading sector: {sector}")
        try:
            df = yf.download(symbols, start=start, end=end, interval=interval, group_by='ticker', progress=False)
            for symbol in symbols:
                if (symbol,) in df.columns:
                    # MultiIndex fallback (rare)
                    sub_df = df[symbol]
                else:
                    # Normal format
                    sub_df = df[symbol].copy()
                sub_df = sub_df.reset_index()
                sub_df["Symbol"] = symbol
                sub_df["Sector"] = sector
                all_data.append(sub_df)
        except Exception as e:
            print(f"Failed to download sector {sector}: {e}")
        time.sleep(5)
        
        
    combined = pd.concat(all_data)
    return combined.reset_index(drop=True)

stock_df = download_by_sector_batch(sectors, start_date, end_date)
stock_df.head()


YF.download() has changed argument auto_adjust default to True



10 Failed downloads:
['AAPL', 'INTC', 'META', 'GOOGL', 'NFLX', 'AMD', 'MSFT', 'AMZN', 'NVDA', 'TSLA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
/tmp/ipykernel_111411/3900817989.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  if (symbol,) in df.columns:
/tmp/ipykernel_111411/3900817989.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  if (symbol,) in df.columns:
/tmp/ipykernel_111411/3900817989.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  if (symbol,) in df.columns:
/tmp/ipykernel_111411/3900817989.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  if (symbol,) in df.columns:
/tmp/ipykernel_111411/3900817989.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  if (symbol,) in df.columns:
/tmp/ipykernel_111411/3900817989.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  if (sy


1 Failed download:
['^GSPC']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Price,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Sector


### Save to CSV

In [4]:
# Save the downloaded dataset
csv_path = "stock_sector_data.csv"
stock_df.to_csv(csv_path, index=False)
print(f"Saved dataset to {csv_path}")


Saved dataset to stock_sector_data.csv
